# 第一題 #
讀取2017所有表特版的文章

In [1]:
import requests
import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import codecs

# function part #

* check_date_from_page
傳進頁面的soup，檢查是否存在12/31號的文章
若有，回傳該文章之soup

* get_website
傳入url，回傳該網站的網站位置

In [2]:
def check_date_from_page(cur_article):
    """
    
    """
    def check_date(soup):
        """
        input beautifuy object, check date
        must find last
        """
        #get date
        date_string=soup.find('div', 'date').string.strip()
        date = datetime.datetime.strptime(date_string,"%m/%d")
        if date.month!=12 or date.day!=31:
            return False
        else:
            return True
        
    cur_article = cur_article.find('div', 'r-ent')
    while cur_article is not None:
        if not check_date(cur_article):
            # next
            cur_article = cur_article.find_next('div', 'r-ent')
        else:
            # find last match
            while cur_article is not None:
                prev_article = cur_article
                cur_article = cur_article.find_next('div', 'r-ent')
                # got 1/1
                if not check_date(cur_article):
                    return prev_article
            return prev_article
    return False
    
def get_website(url):
    """
    get the website from url 
    """
    site = urlparse(url)
    site_netloc = site.scheme + '://' + site.netloc
    return site_netloc

def get_response(url):
    response = requests.get(url)
    while not response.ok:
        response = requests.get(url)
    return response

def get_prev_page_url(root_soup):
    return  website + root_soup.find('div', id='action-bar-container').find('div', 'btn-group-paging').find_all('a')[1]['href']

* 沒網址的不用爬，可忽略
* 爬2017年的所有文章
* 忽略分類為公告的文章
* output:
* 1. 所有文章 
  2. 所有爆文
* 檔案內容 - 日期,標題,url（無空格

## 結束條件 ##
https://www.ptt.cc/bbs/Beauty/M.1483208018.A.411.html

# main function start #

In [3]:
# find the 12/31 article
# END IN this post
END_POST = '[正妹] 瘦身日本妹'
url = "https://www.ptt.cc/bbs/Beauty/index.html"
# get ptt's website
website = get_website(url)
# first flag
first_flag = True
############### loading ####################
response = get_response(url)
root_soup = BeautifulSoup(response.text, 'lxml')
############### end loading ################
find_soup = check_date_from_page(root_soup)

while not find_soup:
    # find next page
    print('page: ' + url + ' not find!')
    print('get next page...')
    # prev page's url
    url = get_prev_page_url(root_soup)
    
    # if this is prist page, than direct minus 300 ...
    if first_flag == True:
        print("first page, direct jump ...")
        # got the number part
        match = re.search(r'(\d+)(?:.html$)', url)
        number = match[1]
        number = int(number) - 300
        # replace back
        url = re.sub(r"(\d+)(.html$)", str(number) + r'\2', url)
        print('jump to: ', url)
        first_flag = False
    
    # loading
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')
    find_soup = check_date_from_page(root_soup)
    
print('got ya')

page: https://www.ptt.cc/bbs/Beauty/index.html not find!
get next page...
first page, direct jump ...
jump to:  https://www.ptt.cc/bbs/Beauty/index2352.html
page: https://www.ptt.cc/bbs/Beauty/index2352.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2351.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2350.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2349.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2348.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2347.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2346.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2345.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2344.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2343.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2342.html not find!
ge

# function for next part of code #


In [4]:
def get_date(soup):
    date_string = soup.find('div', 'date').string.strip()
    return date_string

def get_title(cur_soup):
    if cur_soup.find('div','title').find('a'):
        title = ''
        for string in cur_soup.find('div', 'title').a.strings:
            #if @ occur, need specical parse
            if string == '[email protected]':
                title = title '@' + string['data-cfemail']
            else:
                title = title + string

        title = cur_soup.find('div','title').a.string.strip()
        return title
    else:
        return False
    
def write_info(all_file, pop_file, soup):
    """soup is on <div r-ent>"""
    def check_title_string(title):
        """title is a string, true mean this is what we need"""
        match = re.search(r'\[公告\]', title)
        if not match:
            return True
        else:
            return False
    
    # check if title.a exist
    if not soup.find('div', 'title').find('a'):
        return False
    #check title contain [公告] or not
    title = get_title(soup)
    if check_title_string(title):
        # grab date, topic, url
        date_temp = soup.find('div', 'date').string.strip()
        # remove /
        date = re.sub(r'[/]', '', date_temp)
        # title
        url = website + soup.find('div', 'title').find('a')['href']
        # merge to one line
        write_line = ','.join([date, title, url])
        # if 爆, write to pop_file
        thumb_number = soup.find('div', 'nrec')
        if thumb_number.find('span'):
            thumb_number = thumb_number.span.string.strip()
        else:
            tuhmb_number = ''
        if thumb_number == '爆':
            pop_file.write(write_line+'\n')
        all_file.write(write_line+'\n')
        # write to all_file



In [5]:
# start soup
# root_soup is current soup on root
start_soup = find_soup
## print(start_soup.find_all_previous('div', 'r-ent'))
## print(type(start_soup.find_all_previous('div', 'r-ent')))
# open file
all_file = codecs.open("all_articles.txt", "w", "utf-8")
pop_file = codecs.open("all_popular.txt", "w", "utf-8")
## f.write("Now the file has one more line!")


# first page is special case 
write_info(all_file, pop_file, start_soup)
for cur_soup in start_soup.find_all_previous('div', 'r-ent'):
    if get_title(cur_soup):
        print('write file for "'+get_title(cur_soup)+'"')
    write_info(all_file, pop_file, cur_soup)

# second page start
is_continued = True
while is_continued:
    # to next page
    url = get_prev_page_url(root_soup)
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')
    
    # loop root_soup
    for cur_soup in reversed(root_soup.find_all('div', 'r-ent')):
        title = get_title(cur_soup)
        if title:
            print('write file for "'+title+'"')
        write_info(all_file, pop_file, cur_soup)
        # ending point
        global END_POST
        if title == END_POST:
            is_continued = False
            break
    is_continued == False
all_file.close()
pop_file.close()
    

write file for "[正妹] 2018 世界新車大展 Day 1"
write file for "[正妹] 2018"
write file for "[正妹] 奶特"
write file for "[正妹] 韓流"
write file for "[正妹] 彩彩小萌獸"
write file for "[正妹] 柏木由紀"
write file for "[正妹] 俄羅斯暗黑女模Helga Lovekaty"
write file for "[正妹] 彩美旬果"
write file for "[正妹] 秀氣長髮"
write file for "[神人] 台北車展Polo beats DJ"
write file for "Re: [神人] hotels.com廣告女主角"
write file for "[正妹] 台北車展lexus正妹"
write file for "[正妹] 韓流女團藝人"
write file for "[神人] 自問自答"
write file for "[神人]台北車展land rover的sg"
write file for "[正妹] 跨年前發一波"
write file for "[帥哥] 最近好像流行這種"
write file for "[正妹] 企業排球聯賽 中纖美女主攻手"
write file for "[正妹] 無題"
write file for "[正妹] 世界第衣美"
write file for "[正妹] 高畑充希 Takahata Mitsuki"
write file for "[正妹] 兩個高三生"
write file for "[神人] 柯批剪綵旁的正妹"
write file for "[正妹] Madison Iseman"
write file for "[正妹] 李元玲"
write file for "[帥哥] 正太控請進"
write file for "[正妹] 浜辺美波"
write file for "[正妹] IU"
write file for "[正妹] 阿旦"
write file for "[正妹] 小松菜奈2017"
write file for "[正妹] 收到來信 想試試水溫的鄉民"
write file for "[正妹] 瘦身日本妹"


In [20]:
text =\
"""
<div class="title">
    <a href="/bbs/Beauty/M.1504367479.A.63D.html">[正妹] 乃木坂<span class="__cf_email__" data-cfemail="093d3f495d4e4a">[email&#160;protected]</span></a>
</div>
"""
print(text)
test = BeautifulSoup(text, 'lxml')
for i in test.a.strings:
    print(i)


<div class="title">
    <a href="/bbs/Beauty/M.1504367479.A.63D.html">[正妹] 乃木坂<span class="__cf_email__" data-cfemail="093d3f495d4e4a">[email&#160;protected]</span></a>
</div>

[正妹] 乃木坂
[email protected]
